In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.kernel_ridge import KernelRidge
import math
import time
from sklearn.metrics import r2_score

In [8]:
def preprocess_features(x1, x2, reversed=False):
    x1_size, x2_size = len(x1), len(x2)
    features = np.array([np.concatenate((x2[j][:-1], x1[i][:-1])) if reversed else np.concatenate((x1[i][:-1], x2[j][:-1])) 
                         for i in range(x1_size) for j in range(x2_size)])   
    labels = np.array([(x2[j][-1] - x1[i][-1]) if reversed else (x1[i][-1] - x2[j][-1]) 
                       for i in range(x1_size) for j in range(x2_size)])
    return features, labels

In [9]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
  
def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.sum(np.abs(y_true - y_pred)) / np.sum(y_pred + y_true)

def root_mean_squared_error(y_true, y_pred):
    return np.sqrt(((y_pred - y_true) ** 2).mean())

def mean_absolute_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs(y_true - y_pred))


In [10]:
def find_yn(z, y_sum, N):
    return np.array([(y_sum + sum(z[i: i + N])) / N for i in range(0, len(z), N)])

In [11]:
def find_yn2(z1, z2, y_sum, N):
    return np.array([(y_sum + (sum(z1[i: i + N]) - sum(z2[i: i + N])) / 2) / N for i in range(0, len(z1), N)])

In [12]:
train_data = np.loadtxt("OBDTrain.txt",delimiter=',')
test_data = np.loadtxt("OBDTest.txt",delimiter=',')


In [13]:
y_sum = sum([a[-1] for a in train_data])
N = len(train_data)
y_target_test = np.array([a[-1] for a in test_data])
y_target_train = np.array([a[-1] for a in train_data])

In [14]:
train_features, train_labels = preprocess_features(train_data, train_data)
train_features2, train_labels2 = preprocess_features(train_data, train_data, reversed=True)
test_features, test_labels = preprocess_features(test_data, train_data)
test_features2, test_labels2 = preprocess_features(test_data, train_data, reversed=True)

In [15]:
scaler = MaxAbsScaler()
scaler.fit(train_features)
train_features = scaler.transform(train_features)
train_features2 = scaler.transform(train_features2)
test_features = scaler.transform(test_features)
test_features2 = scaler.transform(test_features2)

In [16]:
from sklearn.svm import SVR
svr = SVR(kernel='rbf', gamma='scale', coef0=0.0, epsilon=0.001, max_iter=-1)

start_time = time.time()
svr.fit(train_features, train_labels)
print("--- %s seconds ---" % (time.time() - start_time))

train_pred_z1 = svr.predict(train_features)
train_pred_z2 = svr.predict(train_features2)

pred_z1 = svr.predict(test_features)
pred_z2 = svr.predict(test_features2)

yn_train = find_yn(train_pred_z1, y_sum, N)
yn_test = find_yn(pred_z1, y_sum, N)

yn_train2 = find_yn2(train_pred_z1, train_pred_z2, y_sum, N)
yn_test2 = find_yn2(pred_z1, pred_z2, y_sum, N)
    
print('Training erros func1:')
print("MAPE: " + str(mean_absolute_percentage_error(y_target_train, yn_train)))
print("RMSE: " + str(root_mean_squared_error(y_target_train, yn_train)))
print("MAE: " + str(mean_absolute_error(y_target_train, yn_train)))
print("R2: " + str(r2_score(y_target_train, yn_train)))
print()
print('Testing errors func1:')
print("MAPE: " + str(mean_absolute_percentage_error(y_target_test, yn_test)))
print("RMSE: " + str(root_mean_squared_error(y_target_test, yn_test)))
print("MAE: " + str(mean_absolute_error(y_target_test, yn_test)))
print("R2: " + str(r2_score(y_target_test, yn_test)))
print()
print()
print('Training erros func2:')
print("MAPE: " + str(mean_absolute_percentage_error(y_target_train, yn_train2)))
print("RMSE: " + str(root_mean_squared_error(y_target_train, yn_train2)))
print("MAE: " + str(mean_absolute_error(y_target_train, yn_train2)))
print("R2: " + str(r2_score(y_target_train, yn_train2)))
print()
print('Testing errors func2:')
print("MAPE: " + str(mean_absolute_percentage_error(y_target_test, yn_test2)))
print("RMSE: " + str(root_mean_squared_error(y_target_test, yn_test2)))
print("MAE: " + str(mean_absolute_error(y_target_test, yn_test2)))
print("R2: " + str(r2_score(y_target_test, yn_test2)))
print()
print()
print()
print()
print()
print()

--- 0.09810900688171387 seconds ---
Training erros func1:
MAPE: 37.52775353627084
RMSE: 4.453318408511998
MAE: 3.7574438408510753
R2: 0.6286745349777887

Testing errors func1:
MAPE: 56.204199381974895
RMSE: 5.744623071208204
MAE: 4.569139727910909
R2: 0.24356526281385127


Training erros func2:
MAPE: 37.52841933712254
RMSE: 4.453308780479925
MAE: 3.757443092977295
R2: 0.6286761405803054

Testing errors func2:
MAPE: 56.205171965290965
RMSE: 5.74469900467552
MAE: 4.569204342297977
R2: 0.24354526529920195








In [17]:
from sklearn.svm import SVR
svr = SVR(kernel='poly', gamma='scale', coef0=0.0, epsilon=0.001, max_iter=-1)
start_time = time.time()
svr.fit(train_features, train_labels)
print("--- %s seconds ---" % (time.time() - start_time))
train_pred_z1 = svr.predict(train_features)
train_pred_z2 = svr.predict(train_features2)

pred_z1 = svr.predict(test_features)
pred_z2 = svr.predict(test_features2)

yn_train = find_yn(train_pred_z1, y_sum, N)
yn_test = find_yn(pred_z1, y_sum, N)

yn_train2 = find_yn2(train_pred_z1, train_pred_z2, y_sum, N)
yn_test2 = find_yn2(pred_z1, pred_z2, y_sum, N)
    
print('Training erros func1:')
print("MAPE: " + str(mean_absolute_percentage_error(y_target_train, yn_train)))
print("RMSE: " + str(root_mean_squared_error(y_target_train, yn_train)))
print("MAE: " + str(mean_absolute_error(y_target_train, yn_train)))
print("R2: " + str(r2_score(y_target_train, yn_train)))
print()
print('Testing errors func1:')
print("MAPE: " + str(mean_absolute_percentage_error(y_target_test, yn_test)))
print("RMSE: " + str(root_mean_squared_error(y_target_test, yn_test)))
print("MAE: " + str(mean_absolute_error(y_target_test, yn_test)))
print("R2: " + str(r2_score(y_target_test, yn_test)))
print()
print()
print('Training erros func2:')
print("MAPE: " + str(mean_absolute_percentage_error(y_target_train, yn_train2)))
print("RMSE: " + str(root_mean_squared_error(y_target_train, yn_train2)))
print("MAE: " + str(mean_absolute_error(y_target_train, yn_train2)))
print("R2: " + str(r2_score(y_target_train, yn_train2)))
print()
print('Testing errors func2:')
print("MAPE: " + str(mean_absolute_percentage_error(y_target_test, yn_test2)))
print("RMSE: " + str(root_mean_squared_error(y_target_test, yn_test2)))
print("MAE: " + str(mean_absolute_error(y_target_test, yn_test2)))
print("R2: " + str(r2_score(y_target_test, yn_test2)))
print()
print()
print()
print()
print()
print()

--- 0.12471222877502441 seconds ---
Training erros func1:
MAPE: 21.8131853871114
RMSE: 3.246948357914098
MAE: 2.4254306522067983
R2: 0.8026041579852525

Testing errors func1:
MAPE: 64.54140041060008
RMSE: 6.709502792487055
MAE: 5.027442044863848
R2: -0.0318796892691291


Training erros func2:
MAPE: 21.813760041631006
RMSE: 3.2469438629885623
MAE: 2.425445107441454
R2: 0.8026047045162712

Testing errors func2:
MAPE: 64.5424993316836
RMSE: 6.709661736526651
MAE: 5.027437552205943
R2: -0.03192857905244817








In [18]:
# 